In [2]:
pip install nltk tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)


In [5]:
import os
import json
from nltk.tokenize import sent_tokenize, word_tokenize
from tqdm import tqdm
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TANPHAT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
import re

def clean_text_for_chunking(text):
    # 1. Thay thế ký tự Unicode lỗi
    replacements = {
        "": "=", "": "-", "": "×", "": "÷", "": "∑",
        "": "±", "": "→", "": "→", " ": " ", "": "<=", "": ">="
    }
    for bad, good in replacements.items():
        text = text.replace(bad, good)

    # 2. Tách dòng và xử lý dòng
    lines = text.splitlines()
    cleaned_lines = []

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # 3. Loại bỏ dòng rời rạc in HOA
        if len(line.split()) < 5 and line.isupper():
            continue

        # 4. Loại bỏ dòng chứa link hoặc biểu mẫu
        if re.search(r"http[s]?://|facebook\.com|Email:|Mật khẩu|Đăng nhập|CCCD|Họ tên", line, re.IGNORECASE):
            continue

        # 5. Chuẩn hóa dấu chấm câu cuối câu (nếu thiếu và không phải danh sách chấm đầu dòng)
        if not re.search(r"[.!?…:]$", line) and not line.startswith("-") and not line.startswith("+"):
            line += "."

        cleaned_lines.append(line)

    # 6. Gộp lại thành văn bản sạch
    return " ".join(cleaned_lines)


In [14]:
INPUT_DIR = "data_clean"
OUTPUT_DIR = "data_clean_verified"

file_count = 0
os.makedirs(OUTPUT_DIR, exist_ok=True)

for folder in os.listdir(INPUT_DIR):
    input_subdir = os.path.join(INPUT_DIR, folder)
    output_subdir = os.path.join(OUTPUT_DIR, folder)
    os.makedirs(output_subdir, exist_ok=True)

    if not os.path.isdir(input_subdir):
        continue

    for fname in os.listdir(input_subdir):
        if not fname.endswith(".txt"):
            continue

        input_path = os.path.join(input_subdir, fname)
        output_path = os.path.join(output_subdir, fname)

        with open(input_path, "r", encoding="utf-8") as f:
            raw_text = f.read()

        cleaned_text = clean_text_for_chunking(raw_text)

        with open(output_path, "w", encoding="utf-8") as f:
            f.write(cleaned_text)

        file_count += 1
        print(f"✅ Cleaned: {folder}/{fname}")

print(f"Đã tạo {file_count} file trong: {OUTPUT_DIR}")

✅ Cleaned: admission_text/De an TS2022 HUS.txt
✅ Cleaned: admission_text/index.php_Home_viewnewsVNU_1185.txt
✅ Cleaned: admission_text/index.php_Home_viewpage_143.txt
✅ Cleaned: admission_text/index.php_Home_viewpage_57.txt
✅ Cleaned: admission_text/XHNV_De an tuyen sinh trinh do dai hoc nam 2022 (kem CV1688).txt
✅ Cleaned: programs_text/home_C1885.txt
✅ Cleaned: programs_text/home_C1963.txt
✅ Cleaned: programs_text/home_C1965.txt
✅ Cleaned: programs_text/home_C2019.txt
✅ Cleaned: programs_text/home_C2020.txt
✅ Cleaned: programs_text/home_C2452.txt
✅ Cleaned: programs_text/home_C2455.txt
✅ Cleaned: programs_text/home_C2456.txt
✅ Cleaned: regulations_text/3626_21.10.2022quy-che-dao-tao-dai-hoc-tai-dhqghn-ap-dung-tu-khoa-qh2022.txt
✅ Cleaned: regulations_text/Quy-chế-ĐTĐH-3626.txt
✅ Cleaned: vnu_text/home_C1700.txt
✅ Cleaned: vnu_text/home_C1701.txt
✅ Cleaned: vnu_text/home_C1702.txt
✅ Cleaned: vnu_text/home_C1703.txt
✅ Cleaned: vnu_text/home_C1704.txt
✅ Cleaned: vnu_text/home_C1705.txt


In [6]:
# ========== CHUNK CONFIG ==========
MIN_CHUNK_LEN = 200
MAX_CHUNK_LEN = 500
INPUT_DIR = "data_clean_verified"
OUTPUT_FILE = "chunks/chunks.jsonl"
os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)

In [7]:
# Tải dữ liệu NLTK (cần cho phân tách câu)
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')

In [8]:
def chunk_text_by_char(text, min_len=200, max_len=500):
    import re
    sentences = re.split(r'(?<=[.!?])\s+', text)
    
    chunks = []
    current = ''
    for sentence in sentences:
        if len(current) + len(sentence) <= max_len:
            current += ' ' + sentence
        else:
            if len(current.strip()) >= min_len:
                chunks.append(current.strip())
            current = sentence
    if len(current.strip()) >= min_len:
        chunks.append(current.strip())
    return chunks

In [9]:
# XỬ LÝ MỘT FILE 
def process_single_file(file_path, category):
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()

    chunks = chunk_text_by_char(text)
    file_name = os.path.basename(file_path)

    result = []
    for i, chunk in enumerate(chunks):
        result.append({
            "source_file": file_name,
            "category": category,
            "chunk_id": i,
            "text": chunk
        })
    return result

In [10]:
#  XỬ LÝ TOÀN BỘ THƯ MỤC 
def process_all_files(input_root, output_file):
    with open(output_file, "w", encoding="utf-8") as out:
        for category in os.listdir(input_root):
            folder_path = os.path.join(input_root, category)
            if not os.path.isdir(folder_path):
                continue

            for fname in tqdm(os.listdir(folder_path), desc=f"📁 {category}"):
                if not fname.endswith(".txt"):
                    continue

                file_path = os.path.join(folder_path, fname)
                chunks = process_single_file(file_path, category)
                for chunk in chunks:
                    out.write(json.dumps(chunk, ensure_ascii=False) + "\n")

    print(f"\nĐã tạo chunk từ {input_root} vào: {output_file}")

In [11]:
process_all_files(INPUT_DIR, OUTPUT_FILE)

📁 wiki: 100%|██████████| 1/1 [00:00<00:00, 1000.31it/s]


Đã tạo chunk từ data_clean_verified vào: chunks/chunks.jsonl


In [12]:
def verify_chunks_stats(chunk_file_path, min_len=200, max_len=500):
    from collections import Counter
    import json

    lengths = []
    short, long, valid = 0, 0, 0
    with open(chunk_file_path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            entry = json.loads(line)
            text_len = len(entry.get("text", ""))
            lengths.append(text_len)
            if text_len < min_len:
                short += 1
            elif text_len > max_len:
                long += 1
            else:
                valid += 1

    print(f"Tổng chunk: {len(lengths)}")
    print(f"Hợp lệ ({min_len}–{max_len}): {valid}")
    print(f"Quá ngắn (<{min_len}): {short}")
    print(f"Quá dài (>{max_len}): {long}")
    print(f"Trung bình: {sum(lengths) // len(lengths)} ký tự")
    print(f"Phân bố độ dài:")
    for key in sorted(Counter([l//100*100 for l in lengths]).items()):
        print(f"  {key[0]}-{key[0]+99}: {key[1]}")


In [13]:
verify_chunks_stats("chunks/chunks.jsonl")


Tổng chunk: 1569
Hợp lệ (200–500): 1448
Quá ngắn (<200): 0
Quá dài (>500): 121
Trung bình: 464 ký tự
Phân bố độ dài:
  200-299: 65
  300-399: 111
  400-499: 1236
  500-599: 140
  600-699: 7
  700-799: 2
  800-899: 1
  900-999: 1
  1000-1099: 3
  1300-1399: 2
  1400-1499: 1
